In [1]:
using PyCall

In [2]:
gym = pyimport("gym")

PyObject <module 'gym' from '/home/d9w/.julia/conda/3/lib/python3.7/site-packages/gym/__init__.py'>

In [3]:
cartpole = gym.make("CartPole-v1")

PyObject <TimeLimit<CartPoleEnv<CartPole-v1>>>

In [4]:
state = cartpole.reset()
action = cartpole.action_space.sample()
next_state, reward, done, _ = cartpole.step(action)

([0.04939737410730551, -0.16508445572993605, 0.03675858373086133, 0.3005931360049499], 1.0, false, Dict{Any,Any}())

In [5]:
typeof(state), typeof(action), typeof(reward), typeof(next_state)

(Array{Float64,1}, Int64, Float64, Array{Float64,1})

In [6]:
replay_buffer_size = Int(1e6)
nb_samples = Int(2e6)
nb_batches = Int(1e4)
batch_size = 50

50

In [7]:
abstract type Buffer end
abstract type Transition end

In [8]:
struct GoodTransition <: Transition
    state::Array{Float64}
    action::Int64
    reward::Float64
    next_state::Array{Float64}
end

In [9]:
struct BadTransition <: Transition
    state::Array
    action
    reward
    next_state::Array
end

In [10]:
t = GoodTransition(state, action, reward, next_state)
tbad = BadTransition(state, action, reward, next_state)

BadTransition([0.04878645848476218, 0.03054578112716652, 0.03682814909862235, -0.0034782683880511325], 0, 1.0, [0.04939737410730551, -0.16508445572993605, 0.03675858373086133, 0.3005931360049499])

In [11]:
tqdm = pyimport("tqdm")

function test_insertion_tqdm(buffer::Buffer, nb_samples::Int, transition::Transition)
    state = cartpole.reset()
    for _ in tqdm.trange(nb_samples)
        append(buffer, transition)
    end
end

function test_sampling_tqdm(buffer::Buffer, nb_batches::Int)
    for _ in tqdm.trange(nb_samples)
        sample(buffer, batch_size)
    end
end

test_sampling_tqdm (generic function with 1 method)

In [12]:
function test_insertion_timev(buffer::Buffer, nb_samples::Int, transition::Transition)
    println("Insertion of ", nb_samples, " samples:")
    @timev for i in 1:nb_samples
        append(buffer, transition)
    end
end

function test_sampling_timev(buffer::Buffer, nb_batches::Int)
    println("Sampling of ", nb_samples, " batches:")
    @timev for i in 1:nb_batches
        sample(buffer, batch_size)
    end
end  

test_sampling_timev (generic function with 1 method)

In [13]:
struct ReplayBuffer1 <: Buffer
    data::Vector{Transition}
    capacity::Int64
end

function ReplayBuffer1(capacity::Int64)
    ReplayBuffer1(Vector{Transition}(undef, 0), capacity)
end

function append(buffer::ReplayBuffer1, t::Transition)
    if length(buffer.data) == buffer.capacity
        popfirst!(buffer.data)
    end
    push!(buffer.data, t)
end

function sample(buffer::ReplayBuffer1, batch_size::Int)
    rand(buffer.data, batch_size)
end

sample (generic function with 1 method)

In [14]:
mutable struct ReplayBuffer2 <: Buffer
    data::Vector{Transition}
    capacity::Int64
    i::Int64
    filled::Bool
end

function ReplayBuffer2(capacity::Int64)
    ReplayBuffer2(Vector{Transition}(undef, capacity), capacity, 1, false)
end

function append(buffer::ReplayBuffer2, t::Transition)
    buffer.data[buffer.i] = t
    buffer.i += 1
    if buffer.i > buffer.capacity
        buffer.filled = true
        buffer.i = 1
    end
end

function sample(buffer::ReplayBuffer2, batch_size::Int)
    if buffer.filled
        println
        return rand(buffer.data, batch_size)
    else
        return rand(buffer.data[1:buffer.i-1], batch_size)
    end
end

sample (generic function with 2 methods)

In [15]:
memory = ReplayBuffer1(replay_buffer_size)
test_insertion_tqdm(memory, nb_samples, t)
test_sampling_tqdm(memory, nb_batches)
test_insertion_timev(memory, nb_samples, t)
test_sampling_timev(memory, nb_batches)

Insertion of 2000000 samples:
  

100%|████████████████████████████| 2000000/2000000 [00:06<00:00, 326536.79it/s]

0.033539 seconds (1 allocation: 0 bytes)
elapsed time (ns): 33539214
realloc() calls:   1
Sampling of 2000000 batches:
  0.020159 seconds (10.00 k allocations: 4.730 MiB)
elapsed time (ns): 20159457
bytes allocated:   4960000
pool allocs:       10000


In [ ]:
memory = ReplayBuffer1(replay_buffer_size)
test_insertion_timev(memory, nb_samples, t)
test_sampling_timev(memory, nb_batches)
test_insertion_tqdm(memory, nb_samples, t)
test_sampling_tqdm(memory, nb_batches)
test_insertion_timev(memory, nb_samples, t)
test_sampling_timev(memory, nb_batches)

In [16]:
# with untyped transitions
memory = ReplayBuffer1(replay_buffer_size)
test_insertion_tqdm(memory, nb_samples, tbad)
test_sampling_tqdm(memory, nb_batches)
test_insertion_timev(memory, nb_samples, tbad)
test_sampling_timev(memory, nb_batches)

Insertion of 2000000 samples:
  0.052615 seconds (1 allocation: 0 bytes)
elapsed time (ns): 52615481
realloc() calls:   1
Sampling of 2000000 batches:
  0.013670 seconds (10.00 k allocations: 4.730 MiB)
elapsed time (ns): 13670004
bytes allocated:   4960000
pool allocs:       10000



100%|████████████████████████████| 2000000/2000000 [00:05<00:00, 354661.69it/s]

In [17]:
memory = ReplayBuffer2(replay_buffer_size)
test_insertion_tqdm(memory, nb_samples, t)
test_sampling_tqdm(memory, nb_batches)
test_insertion_timev(memory, nb_samples, t)
test_sampling_timev(memory, nb_batches)

Insertion of 2000000 samples:
  0.003771 seconds
elapsed time (ns): 3770681
Sampling of 2000000 batches:
  0.018892 seconds (10.00 k allocations: 4.730 MiB)
elapsed time (ns): 18892422
bytes allocated:   4960000
pool allocs:       10000



100%|████████████████████████████| 2000000/2000000 [00:06<00:00, 330206.92it/s]

In [ ]:
# with untyped transitions
memory = ReplayBuffer2(replay_buffer_size)
test_insertion_tqdm(memory, nb_samples, tbad)
test_sampling_tqdm(memory, nb_batches)
test_insertion_timev(memory, nb_samples, tbad)
test_sampling_timev(memory, nb_batches)